In [8]:
import sqlalchemy
import pandas as pd 
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3
import pyodbc
import sys 








refresh_token = "AQB9YlZZiYIjSapkbiiCwlmegWWWjzvoIen-XyPC34yDoomEUdJ9WfiO4_vpjqzUaW3K3ck4fpOD0-LBsKTqce9X77HwYlzkOfV-BCO1s7WDg4HuPNsWLfUziLIs_QjQM6E"
base_64 = "ZWY3ZTBlODU5ODVmNGRjZjhiNzVjODNmNzA1ODcxNjQ6OTBmNjkwNWIxMTI3NDgxYzhiMDUzMjEyMjQ2ZGQ1MTE="

# Requests a new updated token every time the file runs
response = requests.post("https://accounts.spotify.com/api/token", 
    data= {"grant_type":"refresh_token","refresh_token":refresh_token},
    headers={"Authorization":"Basic "+ base_64})

token_refresher = response.json()
TOKEN = token_refresher["access_token"]
headers = {
    "Accept" : "application/json",
    "Content-Type" : "application/json",
    "Authorization" : "Bearer {token}".format(token=TOKEN)
}



# Convert time to Unix timestamp in miliseconds      
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

# Download all songs you've listened to "after yesterday", which means in the last 24 hours      
r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = headers)

data = r.json()





song_names = []
album_name = []
artist_names = []
played_at = []
timestamps = []

for song in data["items"]:
    song_names.append(song["track"]["name"])
    album_name.append(song["track"]["album"]["name"])
    artist_names.append(song["track"]["artists"][0]["name"])
    played_at.append(song["played_at"])
    timestamps.append(song["played_at"][0:10])


song_df = pd.DataFrame(list(zip(song_names, album_name, artist_names, played_at, timestamps)), 
        columns =['song_names', 'album_name', "artist_names", "played_at", "timestamps"])


#check if spotify was used yesterday
if song_df.empty:
    print("No songs were played yesterday")
    sys.exit()

#check if the primary key constraint is respected
if song_df["played_at"].is_unique is True:
    pass
else:
    raise Exception("Duplicates are present")

#check if there are NULL values
if song_df.isnull().values.any():
    raise Exception("Nulls are present")
else:
    pass

#deletes possible rows with a different date than yesterday
for e, date in enumerate(song_df["timestamps"]):
    if datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%m-%d") != yesterday.strftime("%Y-%m-%d"):
        song_df = song_df.drop(e)





#connecting to local MS server database
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=localhost;"
                        "Database=spotify_DB;"
                        "trusted_connection=YES;"
                        )


cursor=conn.cursor()
#creating the table
sql_query = """IF OBJECT_ID('dbo.spotify_songs_TB', 'U') IS NULL CREATE TABLE dbo.spotify_songs_TB(
    song_names VARCHAR(200),
    album_name VARCHAR(200),
    artist_names VARCHAR(200),
    played_at DATETIME,
    timestamps DATE,
    CONSTRAINT primary_key_constraint PRIMARY KEY (played_at));"""
cursor.execute(sql_query)

#inserting the values in the table
try:
    for index, row in song_df.iterrows():
        cursor.execute("INSERT INTO dbo.spotify_songs_TB (song_names, album_name, artist_names, played_at, timestamps) values(?,?,?,?,?)",row.song_names, row.album_name, row.artist_names, row.played_at, row.timestamps)
    conn.commit()
    conn.close() 
except:
    conn.commit()
    display(pd.read_sql_query("select * from spotify_songs_TB", conn))
    conn.close() 